In [ ]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

TensorFlow 1.x selected.


In [ ]:
#imports
import pandas as pd
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv") #traning data
test=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv")  #testing data

In [ ]:
train_X=train['text'] #training text
test_X=test['text'] #testing text

In [ ]:
embed_size = 200 # how big is each word vector
maxlen = 70 # max number of words in a question to use

In [ ]:

## Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_X)+list(test_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)
word_index = tokenizer.word_index

In [ ]:
max_features = len(word_index) # how many unique words to use (i.e num rows in embedding vector)

In [ ]:
train_X =pad_sequences(train_X, maxlen=maxlen)  #padding training text to length=70
test_X = pad_sequences(test_X, maxlen=maxlen)  #padding testing text to length=70
train_Y=train['task_2']    #training label
test_Y=test['task_2']       #testing label
train_Y=np_utils.to_categorical(train_Y)   #one-hot encoded training label
test_Y=np_utils.to_categorical(test_Y)   #one-hot encoded testing label

In [ ]:
def load_glove(word_index):
    EMBEDDING_FILE = '/content/drive/My Drive/minor/glove/glove.twitter.27B.200d.txt'   #glove file
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))     #open glove embediing file
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197   #embedding mean, standard deviation for intializing not found vectors
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word) #get vector for ith word
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())  #if vector for lowercase notfound then try for uppercase
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix 

In [ ]:
embedding_matrix = load_glove(word_index)  #create embedding matrix using glove

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5] #varous filter sizes
    num_filters = 36

    inp = Input(shape=(maxlen,))  #input layer
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x) #reshape to make 3d data for conv2d layers.

    maxpool_pool = [] #list of layers using different filter sizes
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   #concat results of all filter sizes
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(3, activation="sigmoid")(z) #output layer

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    print(model.summary())
    
    return model

In [ ]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))  #input layer
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool]) #concat the two poolings
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc) #output layer
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
def model_gru_du(embedding_matrix):
    inp = Input(shape=(maxlen,)) #input layer
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool]) #concat the ouput of two pooling layers
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(3, activation="sigmoid")(conc) #output layer
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
model1=model_cnn(embedding_matrix) #create cnn model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 70, 200)      2492000     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 70, 200, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 70, 1, 36)    7236        reshape_1[0][0]                  
__________

In [ ]:
model2=model_lstm_du(embedding_matrix) #create bilstm model

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 70, 200)      2492000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 70, 256)      337920      embedding_2[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 256)          0           bidirectional_1[0][0]            
____________________________________________________________________________________________

In [ ]:
model3=model_gru_du(embedding_matrix) #create bigru model

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 70, 200)      2492000     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 70, 256)      253440      embedding_3[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_2 (Glo (None, 256)          0           bidirectional_2[0][0]            
____________________________________________________________________________________________

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_X,train_Y, test_size=0.15, random_state=42) #splitting training and validation data

In [ ]:
cp1=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)
history1=model1.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp1])  #training cnn with checkpoint

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1921 samples, validate on 340 samples
Epoch 1/7
1921/1921 [==============================] - 7s 4ms/step - loss: 0.9778 - acc: 0.4977 - val_loss: 0.9007 - val_acc: 0.5471

Epoch 00001: val_acc improved from -inf to 0.54706, saving model to model_cnn.hdf5
Epoch 2/7
1921/1921 [==============================] - 1s 284us/step - loss: 0.7174 - acc: 0.6616 - val_loss: 0.8204 - val_acc: 0.6559

Epoch 00002: val_acc improved from 0.54706 to 0.65588, saving model to model_cnn.hdf5
Epoch 3/7
1921/1921 [==============================] - 1s 290us/step - loss: 0.4318 - acc: 0.8293 - val_loss: 0.8346 - val_acc: 0.6618

Epoch 00003: val_acc improved from 0.65588 to 0.66176, saving model to model_cnn.hdf5
Epoch 4/7
1921/1921 [==============================] - 1s 296us/step - loss: 0.2270 - acc: 0.9381 - val_loss: 0.8641 - val_acc: 0.6676

Epoch 00004: val_acc improved from 0.66176 to 0.66765, saving

In [ ]:
cp2=ModelCheckpoint('model_lstm.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True)
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #training bilstm with checkpoint

Train on 1921 samples, validate on 340 samples
Epoch 1/7
1921/1921 [==============================] - 1s 724us/step - loss: 0.9807 - accuracy: 0.5346 - val_loss: 0.8529 - val_accuracy: 0.6500

Epoch 00001: val_accuracy improved from -inf to 0.65000, saving model to model_lstm.hdf5
Epoch 2/7
1921/1921 [==============================] - 1s 519us/step - loss: 0.7705 - accuracy: 0.6720 - val_loss: 0.7919 - val_accuracy: 0.6618

Epoch 00002: val_accuracy improved from 0.65000 to 0.66176, saving model to model_lstm.hdf5
Epoch 3/7
1921/1921 [==============================] - 1s 518us/step - loss: 0.5831 - accuracy: 0.7569 - val_loss: 0.8263 - val_accuracy: 0.6618

Epoch 00003: val_accuracy did not improve from 0.66176
Epoch 4/7
1921/1921 [==============================] - 1s 513us/step - loss: 0.3928 - accuracy: 0.8402 - val_loss: 1.0715 - val_accuracy: 0.6676

Epoch 00004: val_accuracy improved from 0.66176 to 0.66765, saving model to model_lstm.hdf5
Epoch 5/7
1921/1921 [====================

In [ ]:
cp3=ModelCheckpoint('model_gru.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True)
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #training bigru with checkpoint

Train on 1921 samples, validate on 340 samples
Epoch 1/7
1921/1921 [==============================] - 1s 721us/step - loss: 0.9737 - accuracy: 0.5429 - val_loss: 0.9646 - val_accuracy: 0.5706

Epoch 00001: val_accuracy improved from -inf to 0.57059, saving model to model_gru.hdf5
Epoch 2/7
1921/1921 [==============================] - 1s 474us/step - loss: 0.8148 - accuracy: 0.6481 - val_loss: 0.8286 - val_accuracy: 0.6794

Epoch 00002: val_accuracy improved from 0.57059 to 0.67941, saving model to model_gru.hdf5
Epoch 3/7
1921/1921 [==============================] - 1s 468us/step - loss: 0.6590 - accuracy: 0.7246 - val_loss: 0.7613 - val_accuracy: 0.6941

Epoch 00003: val_accuracy improved from 0.67941 to 0.69412, saving model to model_gru.hdf5
Epoch 4/7
1921/1921 [==============================] - 1s 481us/step - loss: 0.4844 - accuracy: 0.8090 - val_loss: 0.8238 - val_accuracy: 0.6471

Epoch 00004: val_accuracy did not improve from 0.69412
Epoch 5/7
1921/1921 [=======================

In [ ]:
pred_y1=model1.predict(test_X) #predict on testing data for cnn
pred_y2=model2.predict(test_X)  #predict on testing data for bilstm
pred_y3=model3.predict(test_X)  #predict on testing data for bigru
pred_y4=(pred_y1+pred_y2+pred_y3)/3   #predict on testing data on basis of avg of probablities of above
pred_y1=np.argmax(pred_y1,axis=1)
pred_y2=np.argmax(pred_y2,axis=1)
pred_y3=np.argmax(pred_y3,axis=1)
pred_y4=np.argmax(pred_y4,axis=1)
pred_y5=[]               #predict what majority among cnn,gru,lstm say
for i in range(pred_y1.size):
  a={}
  a[0]=0
  a[1]=0
  a[2]=0
  a[pred_y1[i]]+=1
  a[pred_y2[i]]+=1
  a[pred_y3[i]]+=1
  if a[0]>a[1] and a[0]>a[2]:
    val=0
  elif a[1]>a[0] and a[1]>a[2]:
    val=1
  else:
    val=2
  pred_y5.append(val)
pred_y5=np.array(pred_y5)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print("METRICS FOR TESTING DATA")
print("CNN")
print(classification_report(test['task_2'],pred_y1))
print("BiLSTM")
print(classification_report(test['task_2'],pred_y2))
print("BiGRU")
print(classification_report(test['task_2'],pred_y3))
print("HYBRID By Probablity")
print(classification_report(test['task_2'],pred_y4))
print("HYBRID By Vote")
print(classification_report(test['task_2'],pred_y5))

METRICS FOR TESTING DATA
CNN
              precision    recall  f1-score   support

           0       0.70      0.65      0.67        93
           1       0.58      0.88      0.70       124
           2       0.54      0.10      0.17        71

    accuracy                           0.61       288
   macro avg       0.60      0.54      0.51       288
weighted avg       0.61      0.61      0.56       288

BiLSTM
              precision    recall  f1-score   support

           0       0.51      0.88      0.65        93
           1       0.70      0.51      0.59       124
           2       0.41      0.21      0.28        71

    accuracy                           0.56       288
   macro avg       0.54      0.53      0.50       288
weighted avg       0.57      0.56      0.53       288

BiGRU
              precision    recall  f1-score   support

           0       0.74      0.80      0.77        93
           1       0.59      0.48      0.53       124
           2       0.25      0.31